# Stackoverflow data processing

## 1. Get data by querying on https://data.stackexchange.com/ using SQL statements

```
WITH FilteredQuestions AS (
    SELECT q.Id, q.Title, q.CreationDate
    FROM Posts q
    JOIN (
        SELECT ParentId
        FROM Posts
        WHERE PostTypeId = 2
        GROUP BY ParentId
        HAVING COUNT(Id) > 3  -- Only questions with more than 3 answers
    ) a ON q.Id = a.ParentId
    WHERE q.PostTypeId = 1
),
NumberedRows AS (
    SELECT 
        fq.Id AS question_id, 
        fq.Title AS question_title, 
        fq.CreationDate AS question_date, 
        a.Id AS answer_id, 
        a.Body AS answer_text, 
        a.Score AS answer_score,
        ROW_NUMBER() OVER (ORDER BY fq.CreationDate DESC) AS row_num
    FROM FilteredQuestions fq
    JOIN Posts a ON a.ParentId = fq.Id
    WHERE a.PostTypeId = 2
)
SELECT question_id, question_title, question_date, answer_id, answer_text, answer_score
FROM NumberedRows
WHERE row_num BETWEEN 250001 AND 300000; -- Change the range since each time query can only return 50000 rows maximum
```

## 2. Merge into 1 csv

In [ ]:
import csv
from collections import defaultdict

files = [f"{i}.csv" for i in range(1, 21)]
output_file = "merged.csv"

question_data = defaultdict(lambda: {"question_title": "", "question_date": "", "answers": []})

for file in files:
    with open(file, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            qid = row["question_id"]
            question_data[qid]["question_title"] = row["question_title"]
            question_data[qid]["question_date"] = row["question_date"]

            # Store answer and score together
            answer_text = row.get("answer_text", "")
            answer_score = row.get("answer_score", "")

for qid in question_data:
    question_data[qid]["answers"].sort(key=lambda x: x[1], reverse=True)  # Sort by score descending
    question_data[qid]["answers"] = question_data[qid]["answers"][:5]  # Keep only the top 5

with open(output_file, "w", newline="", encoding="utf-8") as f:
    fieldnames = ["question_id", "question_title", "question_date"]
    
    for i in range(1, 6):
        fieldnames.extend([f"answer_{i}", f"answer_{i}_score"])

    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for qid, data in question_data.items():
        row = {
            "question_id": qid,
            "question_title": data["question_title"],
            "question_date": data["question_date"]
        }
        
        for i, (answer, score) in enumerate(data["answers"], start=1):
            row[f"answer_{i}"] = answer
            row[f"answer_{i}_score"] = score

        writer.writerow(row)

print("Merge complete: merged.csv")

## 3. Import data to ```PostgreSQL```

In [ ]:
import psycopg2
from concurrent.futures import ThreadPoolExecutor

# PostgreSQL Connection Function (Each thread needs its own connection)
def get_connection():
    return psycopg2.connect(
        dbname="questions",
        user="ml_admin",
        password="ml#web_db#2224",
        host="ml-web.postgres.database.azure.com",
        port="5432"
    )

# Function to insert a batch
def insert_batch(batch):
    try:
        conn = get_connection()
        cur = conn.cursor()
        sql = f"INSERT INTO question ({', '.join(expected_headers)}) VALUES ({', '.join(['%s'] * len(expected_headers))})"
        cur.executemany(sql, batch)
        conn.commit()
        cur.close()
        conn.close()
        print(f"Inserted {len(batch)} records.")
    except Exception as e:
        print(f"Batch insert failed: {e}")

# Read CSV Data
with open("data.csv", "r", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    headers = next(reader)  
    
    expected_headers = [
        "question_id", "question_title", "question_date", "answer_1", "answer_1_score",
        "answer_2", "answer_2_score", "answer_3", "answer_3_score",
        "answer_4", "answer_4_score", "answer_5", "answer_5_score"
    ]

    batch_data = [
        [int(row[0])] + [None if val == "" else val for val in row[1:]] 
        for row in reader
    ]
    print(f"Total records to insert: {len(batch_data)}")

# Split into smaller batches
batch_size = 5000
batches = [batch_data[i:i + batch_size] for i in range(0, len(batch_data), batch_size)]

# Use ThreadPoolExecutor for parallel insertion
num_threads = min(4, len(batches))  
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(insert_batch, batches)

print("Insertion completed with multithreading!")


Total records to insert: 204219
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 5000 records.
Inserted 4219 records.
Inserted 5000 records.
Insertion completed with 

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')

In [2]:
df.head(10)

,question_id,question_title,question_date,answer_1,answer_1_score,answer_2,answer_2_score,answer_3,answer_3_score,answer_4,answer_4_score,answer_5,answer_5_score
0,79423052,How can I check whether a character is at leas...,2025-02-08 11:06:56,<p>The main problem with your code is that you...,2,<p>Your current replacement length trick is on...,0.0,<p>This program takes a string as input and ch...,0.0,<p>Here is the solution if you don't want to u...,0.0,NaN,NaN
1,79421105,Decent ways to handle malloc failure?,2025-02-07 13:50:34,<p>You can pass <code>NULL</code> to <code>fre...,8,<p>This has been discussed here many times bef...,6.0,"<p>You could use, what I would call 'telescope...",2.0,<p>Given that there's not much you can do once...,2.0,<p>A simple way is to use loops for an array o...,1.0
2,79419690,"How to calculate time difference in minutes, i...",2025-02-07 00:11:58,<p>To calculate the difference in minutes betw...,2,"<p>If these are just times, not datetimes, the...",2.0,"<p>Based on the last sentence, I guess you've ...",0.0,"<p>Use the function =ABS(C3-C2)*1440 in D2, an...",0.0,NaN,NaN
3,79419480,Using Python to replace triple double quotes w...,2025-02-06 22:12:50,"<p>That file looks to be in <a href=""https://d...",3,"<p>Judging by the OP's code, I guess the aim a...",1.0,"<p>You can use:</p>\n<pre class=""lang-py prett...",0.0,<p>The built-in <em>csv</em> module can handle...,0.0,NaN,NaN
4,79419477,Python's regular expression: Non greedy option...,2025-02-06 22:10:59,"<p>Your first group isn't non-greedy, it's onl...",3,<p>Try:</p>\n<pre><code>^((?! Com) (?:(?! Com)...,1.0,<p><strong>The issue that I see is with the sp...,1.0,<p>Group 2 of your regex <code>^( .+?)?( Com:....,0.0,NaN,NaN
5,79419350,Calculate mean time of day in %H:%M:%S in R,2025-02-06 21:05:35,<p>If you have presumed that all the times ins...,2,<p>You could also do it with <code>lubridate</...,1.0,<p>Add the day to your data.</p>\n<pre><code>d...,1.0,<p>You don't talk about dates. It makes a hug...,1.0,NaN,NaN
6,79419142,How to delimit and print third value for every...,2025-02-06 19:36:38,<p>Using any awk if your fields are tab-separa...,12,<p>Tweaking OP's current code to print every 3...,4.0,<p><em><strong>1st solution:</strong></em> Wit...,4.0,<p>a non-awk solution</p>\n<pre><code>$ sed 1q...,3.0,"<p><a href=""https://en.wikipedia.org/wiki/KISS...",2.0
7,79418728,Should I use <form> when I use <mat-form-field...,2025-02-06 16:50:55,<p>it depends! It depends on what you need.</p...,0,<p>Mat form field requires form element to be ...,0.0,<p>There really is no absolute &quot;best prac...,0.0,"<p>If you want access to the <a href=""https://...",0.0,NaN,NaN
8,79418693,RegEx match blocks of text and return those th...,2025-02-06 16:36:21,"<p>Since the start and end is known, just have...",1,<h3>EDIT: This doesn't solve the OP's problem....,0.0,<p>The regex I would use not assuming anything...,0.0,"<h1><a href=""https://regex101.com/r/w4uLG7/2"" ...",0.0,NaN,NaN
9,79418616,How to find all branches with merge conflicts?,2025-02-06 16:12:07,<blockquote>\n<p>But in lieu of doing a dry ru...,2,<p>git-merge-tree(1) does an in-memory merge a...,2.0,<p>You could take a look at what <code>git mer...,0.0,<p>The definition of &quot;conflict&quot; depe...,0.0,NaN,NaN
